<a href="https://colab.research.google.com/github/Talida-M/AWBD_Project/blob/main/SQL_Injection_Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3

# Set up an in-memory SQLite database (for Google Colab environment)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create users table with additional fields for demonstration
cursor.execute('''CREATE TABLE users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL,
                    password TEXT NOT NULL,
                    email TEXT NOT NULL,
                    role TEXT NOT NULL)''')

# Insert some initial users
cursor.execute("INSERT INTO users (username, password, email, role) VALUES ('admin', 'adminpass', 'admin@example.com', 'admin')")
cursor.execute("INSERT INTO users (username, password, email, role) VALUES ('user1', 'user1pass', 'user1@example.com', 'user')")
cursor.execute("INSERT INTO users (username, password, email, role) VALUES ('user2', 'user2pass', 'user2@example.com', 'user')")
conn.commit()

# Vulnerable register function
def register_user_vulnerable(username, password, email, role):
    query = f"INSERT INTO users (username, password, email, role) VALUES ('{username}', '{password}', '{email}', '{role}')"
    cursor.execute(query)
    conn.commit()
    return "User registered successfully!"

# Secure register function (using parameterized queries)
def register_user_secure(username, password, email, role):
    query = "INSERT INTO users (username, password, email, role) VALUES (?, ?, ?, ?)"
    cursor.execute(query, (username, password, email, role))
    conn.commit()
    return "User registered successfully!"

# Vulnerable login function (SQL Injection susceptible)
def login_vulnerable(username, password):
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)
    result = cursor.fetchone()

    if result:
        return f"Login successful! Welcome, {result[1]}."
    else:
        return "Invalid credentials!"

# Secure login function (using parameterized queries)
def login_secure(username, password):
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
        return f"Login successful! Welcome, {result[1]}."
    else:
        return "Invalid credentials!"

# Vulnerable user listing function
def list_users_vulnerable():
    query = "SELECT * FROM users"
    cursor.execute(query)
    result = cursor.fetchall()

    return result

# Secure user listing function
def list_users_secure():
    query = "SELECT * FROM users"
    cursor.execute(query)
    result = cursor.fetchall()

    return result

# Vulnerable update password function
def update_password_vulnerable(username, new_password):
    query = f"UPDATE users SET password = '{new_password}' WHERE username = '{username}'"
    cursor.execute(query)
    conn.commit()
    return f"Password for {username} updated successfully!"

# Secure update password function
def update_password_secure(username, new_password):
    query = "UPDATE users SET password = ? WHERE username = ?"
    cursor.execute(query, (new_password, username))
    conn.commit()
    return f"Password for {username} updated successfully!"

# Demonstrating SQL Injection
def demonstrate_sql_injection():
    print("---- SQL Injection Demonstration ----\n")

    # 1. Testing the vulnerable login function
    print("Attempting login with SQL Injection:")
    print(login_vulnerable("admin' OR 1=1 --", "any_password"))  # Expected: Login successful!

    # 2. Demonstrating SQL Injection in the user listing function
    print("\nListing all users with SQL Injection:")
    print(list_users_vulnerable())  # List all users (normally)

    # 3. Attacker attempts to list all users using SQL injection (e.g., exploiting vulnerable function)
    print("\nAttacker listing users by bypassing with SQL Injection:")
    print(list_users_vulnerable())  # SQL Injection here isn't as obvious in a list operation, but we'll simulate an attack through a wider vulnerability in app

    # 4. Using SQL Injection to update password
    print("\nAttempting to change password with SQL Injection:")
    print(update_password_vulnerable("user1' --", "newpassword123"))  # Expected: Password for user1 updated successfully!

    # 5. Testing secure login function (no injection allowed)
    print("\nTesting secure login (no SQL Injection):")
    print(login_secure("admin' OR 1=1 --", "any_password"))  # Expected: Invalid credentials

    # 6. Testing secure user listing function
    print("\nTesting secure user listing (no SQL Injection):")
    print(list_users_secure())  # Should list users without issue

    # 7. Testing secure password update
    print("\nTesting secure password update:")
    print(update_password_secure("user2", "securepassword456"))  # Expected: Password for user2 updated successfully!

# Run the demonstration
demonstrate_sql_injection()

# Close the connection
conn.close()


---- SQL Injection Demonstration ----

Attempting login with SQL Injection:
Login successful! Welcome, admin.

Listing all users with SQL Injection:
[(1, 'admin', 'adminpass', 'admin@example.com', 'admin'), (2, 'user1', 'user1pass', 'user1@example.com', 'user'), (3, 'user2', 'user2pass', 'user2@example.com', 'user')]

Attacker listing users by bypassing with SQL Injection:
[(1, 'admin', 'adminpass', 'admin@example.com', 'admin'), (2, 'user1', 'user1pass', 'user1@example.com', 'user'), (3, 'user2', 'user2pass', 'user2@example.com', 'user')]

Attempting to change password with SQL Injection:
Password for user1' -- updated successfully!

Testing secure login (no SQL Injection):
Invalid credentials!

Testing secure user listing (no SQL Injection):
[(1, 'admin', 'adminpass', 'admin@example.com', 'admin'), (2, 'user1', 'newpassword123', 'user1@example.com', 'user'), (3, 'user2', 'user2pass', 'user2@example.com', 'user')]

Testing secure password update:
Password for user2 updated successfull

In [1]:
import sqlite3

# Set up an in-memory SQLite database (for Google Colab environment)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create users table
cursor.execute('''CREATE TABLE users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL,
                    password TEXT NOT NULL)''')

# Insert a test user into the table
cursor.execute("INSERT INTO users (username, password) VALUES ('admin', 'password123')")
conn.commit()

# Vulnerable login function (SQL Injection susceptible)
def login_vulnerable(username, password):
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)
    result = cursor.fetchone()

    if result:
        return "Login successful!"
    else:
        return "Invalid credentials!"

# Secure login function (using parameterized queries)
def login_secure(username, password):
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
        return "Login successful!"
    else:
        return "Invalid credentials!"

# Test the vulnerable login (SQL Injection demonstration)
def demonstrate_sql_injection():
    # Normal login
    print("Normal login with valid credentials:")
    print(login_vulnerable("admin", "password123"))  # Expected: Login successful!

    print("\nAttempting SQL Injection login:")
    # SQL Injection: ' OR 1=1 -- will bypass authentication
    print(login_vulnerable("admin' OR 1=1 --", "any_password"))  # Expected: Login successful!

    # Test the secure login function to show the difference
    print("\nTesting secure login with the same input:")
    print(login_secure("admin' OR 1=1 --", "any_password"))  # Expected: Invalid credentials

# Run the demonstration
demonstrate_sql_injection()

# Close the connection
conn.close()


Normal login with valid credentials:
Login successful!

Attempting SQL Injection login:
Login successful!

Testing secure login with the same input:
Invalid credentials!
